# Lesson 3: Relationships & Items

## The Problem: Items on Receipts

So far, our receipt system has a fundamental limitation: we can track the grand total, but we have no idea what was actually purchased. All the important details—the shampoo, the toothpaste, the gallon of milk—are invisible to our database.

```python
# We only know the total, not what was bought!
receipt = Receipt(merchant="Target", grand_total=49.67)
# But what did we buy? We have no way to know!
```

This creates several problems. First, we can't itemize a receipt. Second, we can't verify that the grand total is correct. And third, we have no way to analyze purchasing patterns across multiple receipts. We need a better approach.

## What We Actually Want

What we really need is the ability to store individual items and associate them with receipts. Each receipt should contain a list of items, and we should be able to calculate the total from those items automatically.

```python
# This is what we want to do:
receipt = Receipt(merchant="Target")
receipt.items.append(Item(name="Shampoo", price=8.99))
receipt.items.append(Item(name="Toothpaste", price=5.49))
receipt.items.append(Item(name="Milk", price=4.29))
receipt.calculate_total()  # Auto-calculate from items!
```

This tells us we need a new table to store items, where each item is linked to a specific receipt.

## Creating the Item Class

To make this work, we need to define an `Item` class that can connect to our `Receipt` class. Here's what that looks like:

```python
class Item(Base):
    __tablename__ = 'items'
    
    id = Column(Integer, primary_key=True)
    receipt_id = Column(Integer, ForeignKey('receipts.id'), nullable=False)
    name = Column(String, nullable=False)
    price = Column(Float, nullable=False)
    quantity = Column(Integer, default=1)
    
    # Relationship: each item belongs to one receipt
    receipt = relationship("Receipt", back_populates="items")
    
    def subtotal(self):
        """Calculate item subtotal"""
        return self.price * self.quantity
    
    def __repr__(self):
        return f"Item({self.name}: ${self.price} x{self.quantity})"
```

## Understanding the receipt_id

Notice the `receipt_id` column in our `Item` class. This is crucial: it tells the database which receipt each item belongs to. Without it, we'd have a collection of orphaned items with no way to know which receipt they came from.

Think of it like a filing system: each item needs a label saying "I belong to receipt #42" so we can group all the items from the same shopping trip together.

```python
# The receipt_id creates the connection:
# Item 1: Shampoo    -> receipt_id = 5
# Item 2: Toothpaste -> receipt_id = 5  
# Item 3: Milk       -> receipt_id = 5
# All three items belong to receipt #5
```

The `ForeignKey('receipts.id')` part tells SQLAlchemy: "This number must match an actual receipt ID in the receipts table." This prevents us from accidentally creating items for receipts that don't exist.

## What Does back_populates Mean?

The `back_populates` parameter creates a two-way relationship between receipts and items. It means:

- From an item, you can access `item.receipt` to get the receipt it belongs to
- From a receipt, you can access `receipt.items` to get all items on that receipt

```python
# Going from item to receipt:
milk = Item(name="Milk", price=4.29)
print(milk.receipt.merchant)  # Access the receipt from the item

# Going from receipt to items:
receipt = Receipt(merchant="Target")
for item in receipt.items:  # Access all items from the receipt
    print(item.name)
```

Without `back_populates`, we'd only be able to navigate in one direction. The "populates" part means SQLAlchemy automatically keeps both sides of the relationship in sync—when you add an item to a receipt, the item's receipt reference is automatically set.

## Understanding Relationships in Databases

When we connect tables together—like linking items to receipts—we create relationships that introduce new complexities. Let's explore the problems that arise and the concepts we need to handle them.

### The Orphan Problem

Consider what happens when you delete a receipt from your database. You've removed the receipt record, but what about all the items that belonged to it? Those items are now orphaned as they reference a receipt that no longer exists. Your database now contains meaningless data: items pointing to receipt ID #42, but receipt #42 is gone.

This creates several issues: the items waste storage space and they clutter your database with invalid data. This is called the **orphan record problem**, and it's one of the most common issues in relational databases.

### Solution: Cascade Deletion

To solve this, we need to tell the database: "When I delete a receipt, automatically delete all items that belong to it." This is called **cascade deletion** or **cascade delete**. It creates a rule that deletions cascade down the relationship—deleting the parent automatically deletes the children.

Without cascade delete, you'd need to manually find and delete all items before deleting the receipt:
```python
# Without cascade delete - tedious and error-prone:
receipt = session.query(Receipt).get(5)
for item in receipt.items:
    session.delete(item)  # Delete each item manually
session.delete(receipt)    # Then delete the receipt
```

With cascade delete configured, the database handles it automatically:
```python
# With cascade delete - clean and safe:
receipt = session.query(Receipt).get(5)
session.delete(receipt)  # Items are automatically deleted too
```

### The Detachment Problem

Here's another scenario: you're editing a receipt and you decide to remove an item from the receipt's item list, but you don't explicitly delete it. What should happen to that item?

```python
receipt = session.query(Receipt).get(5)
milk = receipt.items[2]  # Get the third item
receipt.items.remove(milk)  # Remove it from the list
# Now what? The milk item still exists in the database, orphaned!
```

The item has been detached from its parent but still exists in the database. This creates the same orphan problem as before. The solution is **delete-orphan**, which tells the database: "If an item is removed from its parent's collection, delete it from the database entirely." This ensures that items can't exist without a receipt.

### Bidirectional Navigation

When you create a relationship between receipts and items, you need to decide: can I navigate in both directions? Do I want to be able to:
- Start with a receipt and access its items (`receipt.items`)
- Start with an item and access its receipt (`item.receipt`)

This is called a **bidirectional relationship**. To make this work, you need to set up the relationship on both sides and tell each side about the other. This is done through **back_populates**, which means "this relationship corresponds to that relationship on the other class."

```python
# Bidirectional navigation:
receipt = session.query(Receipt).get(5)
first_item = receipt.items[0]  # Navigate from receipt to item
parent_receipt = first_item.receipt  # Navigate back from item to receipt
```

Without back_populates, you could only navigate in one direction, making your code more cumbersome and queries more complex.

### Foreign Keys: The Foundation

None of these relationships work without a **foreign key**. A foreign key is a column in one table that stores the ID of a record in another table. In our case, each item has a `receipt_id` column that stores which receipt it belongs to.

The foreign key serves two purposes: it creates the link between tables (allowing the database to know which items belong to which receipt), and it enforces referential integrity (preventing you from creating items that reference non-existent receipts).

```python
# The foreign key creates the connection:
item = Item(name="Milk", price=4.29, receipt_id=5)
# This item now belongs to receipt #5
```





### Example Prompts for LLMs

Now that we've covered these concepts, here are example prompts you can use:

**Basic relationship setup:**
```
Create two SQLAlchemy classes: Receipt and Item. Each receipt can have 
multiple items. Add a foreign key in Item that references Receipt. 
Make the relationship bidirectional using back_populates.
```

**Adding cascade deletion:**
```
When I delete a receipt, I want all its items to be automatically deleted. 
Add cascade delete to the relationship.
```

**Handling orphans:**
```
In my Receipt-Item relationship, if I remove an item from a receipt's 
item list, I want that item to be deleted from the database entirely. 
Add delete-orphan to prevent orphaned items.
```

**Complete setup:**
```
Create a Receipt and Item database model where:
- One receipt has many items
- Items have a foreign key linking to receipts
- The relationship is bidirectional (I can navigate from receipt to items 
  and from item to receipt)
- Deleting a receipt automatically deletes all its items
- Removing an item from a receipt's list deletes it from the database
```

The key is understanding what problems you're solving, not just memorizing what to ask for. When you know these issues exist, you can describe them to the LLM in plain English and it will implement the technical solution.

In [ ]:
### Creating the Item Class
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import date

engine = create_engine('sqlite:///receipts.db')
Base = declarative_base()

class Receipt(Base):
    __tablename__ = 'receipts'
    
    id = Column(Integer, primary_key=True)
    merchant = Column(String, nullable=False)
    phone = Column(String)
    total = Column(Float)
    tax = Column(Float)
    grand_total = Column(Float)
    date = Column(Date, default=date.today)
    
    # Relationship: one receipt has many items
    items = relationship("Item", back_populates="receipt", cascade="all, delete-orphan")
    
    def calculate_total(self):
        """Calculate total from items"""
        self.total = sum(item.price * item.quantity for item in self.items)
        self.tax = self.total * 0.045  # 4.5% tax rate
        self.grand_total = self.total + self.tax
        return self.grand_total
        
    def __repr__(self):
        return f"Receipt #{self.id}: {self.merchant} - ${self.grand_total:.2f}"

class Item(Base):
    __tablename__ = 'items'
    
    id = Column(Integer, primary_key=True)
    receipt_id = Column(Integer, ForeignKey('receipts.id'), nullable=False)
    name = Column(String, nullable=False)
    price = Column(Float, nullable=False)
    quantity = Column(Integer, default=1)
    
    # Relationship: each item belongs to one receipt
    receipt = relationship("Receipt", back_populates="items")
    
    def subtotal(self):
        """Calculate item subtotal"""
        return self.price * self.quantity
    
    def __repr__(self):
        return f"Item({self.name}: ${self.price} x{self.quantity})"

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)


### Working with Related Objects

**Creating Receipt with Items:**


In [ ]:
session = Session()

In [ ]:
receipt = Receipt(merchant="Target", phone="555-0200")


In [ ]:
receipt.items.append(Item(name="Shampoo", price=8.99, quantity=1))
receipt.items.append(Item(name="Toothpaste", price=5.49, quantity=1))
receipt.items.append(Item(name="Milk", price=4.29, quantity=1))
receipt.items.append(Item(name="Bread", price=3.50, quantity=1))


In [ ]:
session.add(receipt)


In [ ]:
receipt.calculate_total()
session.commit()


In [ ]:
receipt.calculate_total()

In [ ]:
(3.50 + 4.29 + 5.49 + 8.99) * 1.045

## Querying with Relationships

Now that we have receipts connected to items, we can ask much more interesting questions. Let's create some sample data to work with.


In [ ]:
from sqlalchemy import func

session = Session()

# Safeway receipt - mostly dairy
safeway1 = Receipt(merchant="Safeway", phone="555-0100")
safeway1.items.append(Item(name="Milk", price=4.29, quantity=2))
safeway1.items.append(Item(name="Cheese", price=6.99, quantity=1))
safeway1.items.append(Item(name="Yogurt", price=5.49, quantity=3))
safeway1.items.append(Item(name="Bread", price=3.50, quantity=1))
safeway1.calculate_total()


In [ ]:
# Another Safeway receipt
safeway2 = Receipt(merchant="Safeway", phone="555-0100")
safeway2.items.append(Item(name="Milk", price=4.29, quantity=1))
safeway2.items.append(Item(name="Eggs", price=4.99, quantity=1))
safeway2.items.append(Item(name="Butter", price=5.29, quantity=1))
safeway2.calculate_total()


In [ ]:
# Target receipt - no dairy
target = Receipt(merchant="Target", phone="555-0200")
target.items.append(Item(name="Shampoo", price=8.99, quantity=1))
target.items.append(Item(name="Toothpaste", price=5.49, quantity=1))
target.items.append(Item(name="Paper Towels", price=12.99, quantity=2))
target.calculate_total()


In [ ]:
# Trader Joe's - mixed items
traderjoes = Receipt(merchant="Trader Joe's", phone="555-0300")
traderjoes.items.append(Item(name="Cheese", price=4.99, quantity=2))
traderjoes.items.append(Item(name="Wine", price=8.99, quantity=1))
traderjoes.items.append(Item(name="Crackers", price=3.99, quantity=1))
traderjoes.calculate_total()

In [ ]:
session.add_all([safeway1, safeway2, target, traderjoes])
session.commit()


Relationships let us ask questions that span multiple tables. Without joins, we'd be stuck querying each table separately and manually connecting the results. With joins, the database does the hard work of combining related data, and we can filter, aggregate, and analyze across our entire dataset.

In [ ]:
session.query(Receipt).count()

In [ ]:
session.query(Receipt).all()

In [ ]:
session.query(Item).count()

In [ ]:
session.query(Item).all()

In [ ]:
rec = session.get(Receipt, 1)
rec

In [ ]:
item_1_receipt_1 = rec.items[0]

In [ ]:
item_1_receipt_1

In [ ]:
item_1_receipt_1.receipt

In [ ]:
rec.items.remove(item_1_receipt_1)

In [ ]:
rec.items

In [ ]:
### Basic Relationship Queries

The simplest way to use relationships is to navigate from one object to its related objects. We already have the connection established—we just need to follow it.


In [ ]:
# Get a receipt and access its items
receipt = session.query(Receipt).filter_by(merchant="Safeway").first()
print(f"\n{receipt.merchant} Receipt #{receipt.id}:")
for item in receipt.items:
    print(f"  {item.name}: ${item.price} x{item.quantity} = ${item.subtotal():.2f}")
print(f"Grand Total: ${receipt.grand_total:.2f}")


This works because of the relationship we defined. When we access `receipt.items`, SQLAlchemy automatically queries the database for all items with that receipt's ID.
### Finding Items Across All Receipts

What if we want to find all purchases of a specific product, regardless of which receipt it's on? all we need to do is query the appropriate table (Item in this case)


In [ ]:
# Find all times we bought milk
milk_items = session.query(Item).filter_by(name="Milk").all()
print(f"\nBought milk {len(milk_items)} times:")
for item in milk_items:
    print(f"  {item.receipt.merchant}: {item.quantity} at ${item.price}")


In [ ]:
# Find all times we bought milk
milk_items = session.query(Item).filter_by(price = 4.29).all()
print(f"\nBought milk {len(milk_items)} times:")
for item in milk_items:
    print(f"  {item.receipt.merchant}: {item.quantity} at ${item.price}")


In [ ]:
# Find all times we bought milk
milk_items = session.query(Item).filter(Item.price < 6).all()
print(f"\nBought milk {len(milk_items)} times:")
for item in milk_items:
    print(f"  {item.receipt.merchant}: {item.quantity} at ${item.price}")


In [ ]:
Notice how we can navigate backwards too: `item.receipt.merchant` goes from the item to its receipt. This is the power of bidirectional relationships.


### Understanding Joins

Now we hit a more complex question: which receipts contain milk? We need to look at receipts, but filter based on their items. This requires a **join**.

A join combines two tables based on their relationship. Think of it like this: we're asking the database to create a temporary combined table where each row contains both a receipt and one of its items, then we filter that combined table.


In [ ]:
receipts_with_milk = session.query(Receipt).join(Item).filter(
    Item.name == "Milk"
).all()
receipts_with_milk



print(f"\nReceipts containing milk: {len(receipts_with_milk)}")
for receipt in receipts_with_milk:
    print(f"  {receipt.merchant} on {receipt.date}")
```

**What's happening here:** 
- `query(Receipt)` says we want Receipt objects back
- `.join(Item)` creates the connection between receipts and items
- `.filter(Item.name == "Milk")` filters to only rows where the item is milk
- We get back Receipt objects, not items, because that's what we queried for

Without the join, we couldn't filter receipts based on item properties—the database wouldn't know how to connect them.

## Inheritance - ServiceReceipt & GroceryReceipt

### The Problem: Different Receipt Types

Not all receipts are the same. A restaurant receipt needs to track the waiter's name and tip amount. A grocery store receipt needs to store your loyalty card number. But both are fundamentally receipts—they have merchants, items, totals, and dates.

```python
# Restaurant receipt has waiter and tip
restaurant = ServiceReceipt(
    merchant="Olive Garden",
    waiter="Sarah",
    tip=10.00
)

# Grocery receipt has loyalty number
grocery = GroceryReceipt(
    merchant="Safeway",
    loyalty_number="1234567890"
)

# But they're BOTH receipts with merchant, items, total, etc.
```

We could create completely separate classes with duplicate code, but that violates a fundamental programming principle: don't repeat yourself. Both receipt types share most of their behavior—only a few fields differ. This is the perfect scenario for inheritance.

### Understanding Inheritance in Databases

Inheritance in object-oriented programming is straightforward: a child class inherits properties and methods from a parent class. But in databases, we need a strategy for storing this hierarchy in tables.

SQLAlchemy offers three inheritance strategies, but we'll use the simplest: **single table inheritance**. This means all receipt types—base receipts, service receipts, and grocery receipts—are stored in one table. A special column called the **discriminator** tells SQLAlchemy which type each row represents.

Think of it like a filing cabinet where all receipts go in one drawer, but each has a label ("service" or "grocery") that tells you what kind it is.

## Implementing Inheritance

### The Base Receipt Class

We start by modifying our `Receipt` class to support inheritance. The key additions are the `type` column and the `__mapper_args__` dictionary.

```python
class Receipt(Base):
    """Base receipt class - all receipts have these fields"""
    __tablename__ = 'receipts'
    
    id = Column(Integer, primary_key=True)
    type = Column(String)  # Discriminator column
    merchant = Column(String, nullable=False)
    phone = Column(String)
    total = Column(Float)
    tax = Column(Float)
    grand_total = Column(Float)
    date = Column(Date, default=date.today)
    
    items = relationship("Item", back_populates="receipt", cascade="all, delete-orphan")
    
    # This enables inheritance
    __mapper_args__ = {
        'polymorphic_identity': 'receipt',
        'polymorphic_on': type
    }
```

The `type` column is our **discriminator column**—it stores a string that identifies which class this receipt belongs to. The `__mapper_args__` dictionary tells SQLAlchemy two critical things:

- `polymorphic_on`: which column contains the type information (our `type` column)
- `polymorphic_identity`: what value to store in that column for this specific class ("receipt" for base receipts)

When you create a `Receipt` object, SQLAlchemy automatically sets `type='receipt'`. When you query, SQLAlchemy reads the `type` column to determine which Python class to instantiate.

### Creating ServiceReceipt

Now we can create specialized receipt types. A service receipt adds two fields: the waiter's name and the tip amount.

```python
class ServiceReceipt(Receipt):
    """Restaurant/service receipt with waiter and tip"""
    __tablename__ = None  # Use parent table
    
    waiter = Column(String)
    tip = Column(Float, default=0.0)
    
    __mapper_args__ = {
        'polymorphic_identity': 'service',
    }
```

Notice `__tablename__ = None`—this tells SQLAlchemy not to create a separate table. Instead, the `waiter` and `tip` columns are added to the `receipts` table. This is how single table inheritance works: all columns from all classes share one table.

The `polymorphic_identity` is set to `'service'`, so when you create a `ServiceReceipt`, the `type` column is automatically set to `'service'`.

### Method Overriding: Custom Calculation Logic

Service receipts need different calculation logic because the tip should be included in the grand total. We override the `calculate_total()` method to implement this.

```python
class ServiceReceipt(Receipt):
    # ... (columns as before)
    
    def calculate_total(self):
        """Override to include tip in grand_total"""
        self.total = sum(item.subtotal() for item in self.items)
        self.tax = self.total * 0.09
        # Grand total includes tip!
        self.grand_total = self.total + self.tax + self.tip
```

This is **method overriding**—we replace the parent class's method with our own version. When you call `calculate_total()` on a `ServiceReceipt`, Python uses this version instead of the base `Receipt` version. The tip is now part of the grand total, which makes sense for restaurant bills.

### Creating GroceryReceipt

Grocery receipts are simpler—they just add a loyalty number field and use the standard total calculation.

```python
class GroceryReceipt(Receipt):
    """Grocery store receipt with loyalty program"""
    __tablename__ = None  # Use parent table
    
    loyalty_number = Column(String)
    
    __mapper_args__ = {
        'polymorphic_identity': 'grocery',
    }
```

No need to override `calculate_total()` here as the base class implementation works fine for groceries.


In [1]:
# Create engine and base (with all your new classes defined)
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import date

engine = create_engine('sqlite:///receipts.db')
Base = declarative_base()


print("Created new database with inheritance support")

Created new database with inheritance support


In [2]:
class Receipt(Base):
    """Base receipt class - all receipts have these fields"""
    __tablename__ = 'receipts'
    
    id = Column(Integer, primary_key=True)
    type = Column(String)  # Discriminator column
    merchant = Column(String, nullable=False)
    phone = Column(String)
    total = Column(Float)
    tax = Column(Float)
    grand_total = Column(Float)
    date = Column(Date, default=date.today)
    
    items = relationship("Item", back_populates="receipt", cascade="all, delete-orphan")
    
    # This enables inheritance
    __mapper_args__ = {
        'polymorphic_identity': 'receipt',
        'polymorphic_on': type
    }

    def calculate_total(self):
        """Calculate total from items"""
        self.total = sum(item.price * item.quantity for item in self.items)
        self.tax = self.total * 0.045  # 4.5% tax rate
        self.grand_total = self.total + self.tax
        return self.grand_total
        
    def __repr__(self):
        return f"Receipt #{self.id}: {self.merchant} - ${self.grand_total:.2f}"

In [3]:
class Item(Base):
    __tablename__ = 'items'
    
    id = Column(Integer, primary_key=True)
    receipt_id = Column(Integer, ForeignKey('receipts.id'), nullable=False)
    name = Column(String, nullable=False)
    price = Column(Float, nullable=False)
    quantity = Column(Integer, default=1)
    
    # Relationship: each item belongs to one receipt
    receipt = relationship("Receipt", back_populates="items")
    
    def subtotal(self):
        """Calculate item subtotal"""
        return self.price * self.quantity
    
    def __repr__(self):
        return f"Item({self.name}: ${self.price} x{self.quantity})"


In [4]:
class ServiceReceipt(Receipt):
    """Restaurant/service receipt with waiter and tip"""
    __tablename__ = None  # Use parent table
    
    waiter = Column(String)
    tip = Column(Float, default=0.0)
    
    __mapper_args__ = {
        'polymorphic_identity': 'service',
    }
    def calculate_total(self):
        """Override to include tip in grand_total"""
        self.total = sum(item.subtotal() for item in self.items)
        self.tax = self.total * 0.09
        # Grand total includes tip!
        self.grand_total = self.total + self.tax + self.tip

In [5]:
class GroceryReceipt(Receipt):
    """Grocery store receipt with loyalty program"""
    __tablename__ = None  # Use parent table
    
    loyalty_number = Column(String)
    
    __mapper_args__ = {
        'polymorphic_identity': 'grocery',
    }

In [6]:
Base.metadata.create_all(engine)

# Create session
Session = sessionmaker(bind=engine)
session = Session()

## Using Inherited Classes

### Creating Different Receipt Types

Now we can create receipts of different types and SQLAlchemy handles the details automatically.


In [7]:
from sqlalchemy import func

session = Session()

# Create a service receipt (restaurant)
restaurant = ServiceReceipt(
    merchant="Olive Garden",
    phone="555-0500",
    waiter="Sarah"
)
restaurant.items.extend([
    Item(name="Pasta", price=15.99, quantity=1),
    Item(name="Salad", price=8.99, quantity=1),
    Item(name="Soda", price=2.99, quantity=2),
])
restaurant.tip = 6.00
restaurant.calculate_total()
session.add(restaurant)
session.commit()

Notice we call `calculate_total()` after setting the tip. This ensures the tip is included in the grand total. The order matters here—if you calculate first and then set the tip, the tip won't be included.

In [10]:
# Create a grocery receipt
grocery = GroceryReceipt(
    merchant="Safeway",
    phone="555-0600",
    loyalty_number="1234567890"
)
grocery.items.extend([
    Item(name="Milk", price=4.29, quantity=1),
    Item(name="Bread", price=3.50, quantity=1),
    Item(name="Eggs", price=6.99, quantity=1),
    Item(name="Apples", price=5.99, quantity=2),
])
grocery.calculate_total()
session.add(grocery)

session.commit()

In [11]:
session.query(Receipt).all()

[Receipt #1: Olive Garden - $39.75,
 Receipt #2: Safeway - $27.96,
 Receipt #3: Safeway - $27.96]


Even though these are different Python classes, they're stored in the same `receipts` table. The `type` column contains `'service'` for the restaurant and `'grocery'` for Safeway.


In [14]:
# Check the discriminator column
for rec in session.query(Receipt).all():
    print(f"Type: {rec.type}, Merchant: {rec.merchant}")

Type: service, Merchant: Olive Garden
Type: grocery, Merchant: Safeway
Type: grocery, Merchant: Safeway


In [ ]:

## Querying Across the Hierarchy

The real power of inheritance shows up when querying. You can query broadly (all receipts) or narrowly (only specific types).

### Querying All Receipts

When you query the base `Receipt` class, you get everything—service receipts, grocery receipts, and plain receipts.


In [15]:
# Get ALL receipts (any type)
all_receipts = session.query(Receipt).all()
print(f"\nAll Receipts: {len(all_receipts)} total")
for r in all_receipts:
    print(f"  {r}")


All Receipts: 3 total
  Receipt #1: Olive Garden - $39.75
  Receipt #2: Safeway - $27.96
  Receipt #3: Safeway - $27.96


SQLAlchemy automatically instantiates the correct class for each row based on the `type` column. A row with `type='service'` becomes a `ServiceReceipt` object, not a plain `Receipt` object.


### Querying Specific Types

To get only service receipts, query the `ServiceReceipt` class directly.


In [18]:
service_receipts = session.query(ServiceReceipt).all()
print(f"\nService Receipts: {len(service_receipts)}")
for r in service_receipts:
    print(f"  {r.merchant}: Waiter={r.waiter}, Tip=${r.tip:.2f}")
    tip_pct = (r.tip / r.total * 100) if r.total > 0 else 0
    print(f"    Tip percentage: {tip_pct:.1f}%")


Service Receipts: 1
  Olive Garden: Waiter=Sarah, Tip=$6.00
    Tip percentage: 19.4%


In [ ]:
SQLAlchemy adds a filter automatically: it only returns rows where `type='service'`. You don't have to manually filter by the discriminator column—the inheritance system handles it.

Similarly for grocery receipts:


In [19]:
# Get ONLY grocery receipts
grocery_receipts = session.query(GroceryReceipt).all()
print(f"\nGrocery Receipts: {len(grocery_receipts)}")
for r in grocery_receipts:
    print(f"  {r.merchant} (Loyalty: {r.loyalty_number})")
    print(f"    Total: ${r.grand_total:.2f}")


Grocery Receipts: 2
  Safeway (Loyalty: 1234567890)
    Total: $27.96
  Safeway (Loyalty: 1234567890)
    Total: $27.96


### Beyond Basic Queries: What's Possible

The examples above show fundamental querying patterns—getting all receipts, filtering by type, and accessing subclass attributes. But SQLAlchemy supports much more sophisticated analysis. Here's what's possible once you understand the core concepts we've covered.

**Aggregations**: You can calculate totals, averages, minimums, maximums, and counts across your data. For example: "What's the total amount I've spent on tips?" or "What's my average grocery bill?" or "How many times have I shopped at Safeway?" These use functions like `func.sum()`, `func.avg()`, `func.count()`, `func.min()`, and `func.max()`.

**Filtering and Comparisons**: Beyond simple equality checks, you can filter on ranges, patterns, and combinations. Questions like "Show me all receipts over $50" or "Find receipts from the last month" or "Which service receipts have tips greater than 20%?" These use comparison operators (`>`, `<`, `>=`, `<=`) and filters on dates, amounts, and percentages.

**Grouping**: You can organize data into categories and perform calculations on each group. For instance: "How much did I spend at each merchant?" or "What's my average tip percentage by restaurant?" This groups receipts by merchant or other attributes and calculates statistics for each group.

**Sorting**: You can order results by any attribute—chronologically, by amount, alphabetically. "Show me my most expensive receipts first" or "List all service receipts by date" use sorting operations.

**Complex Combinations**: You can combine all these operations. "For each grocery store, show me my total spending and number of visits, but only for stores where I've spent more than $100 total, sorted by spending" mixes grouping, aggregation, filtering, and sorting.

The key insight: once you understand relationships, joins, filtering, and inheritance—the concepts we've covered—you can describe what you want in plain English to a language model. You don't need to memorize every SQLAlchemy function. Instead, focus on articulating your question clearly: "Calculate the total tips I've given" or "Find my average spending at restaurants versus grocery stores" or "Show me which loyalty card I've used most often."

An LLM can translate these questions into the appropriate SQLAlchemy code. Your job is understanding what's possible and what you're trying to accomplish. The LLM handles the syntax details.



## The Key Benefits of Inheritance

Inheritance in databases gives you several advantages:

**Code reuse**: Common fields and methods are defined once in the base class. You don't duplicate the `merchant`, `total`, or `items` relationship in every subclass.

**Polymorphic queries**: You can query all receipts together or filter to specific types. This flexibility makes your code more powerful without adding complexity.

**Type-specific behavior**: Each subclass can override methods to implement custom logic. Service receipts calculate totals differently than grocery receipts, but both use the same interface.

**Maintainability**: Changes to common functionality only need to be made in one place. If you modify how `calculate_total()` works in the base class, all subclasses inherit the change unless they override it.

The inheritance system lets you model real-world hierarchies naturally while keeping your database schema simple. All receipts live in one table, but your Python code treats them as distinct types with different behaviors.